In [1]:
import os
import ROOT
from math import pow, sqrt

Welcome to JupyROOT 6.28/04


#### Calculate normalization factor for fake rate measurement
#### Based on M(ll) distribution

In [2]:
WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"
ERA     = "2016preVFP"
CHANNEL = "MeasFakeMu"
HLTPATH = "Mu17"
ID      = "tight"

In [3]:
#### sample list
DataStream = ""
if "El" in CHANNEL:
    if "2016" in ERA:  DataStream = "DoubleEG"
    if "2017" in ERA:  DataStream = "SingleElectron"
    if "2018" in ERA:  DataStream = "EGamma"
if "Mu" in CHANNEL:
    DataStream = "DoubleMuon"

In [11]:
W  = ["WJets_MG"]
DY = ["DYJets", "DYJets10to50_MG"]
#DY = ["DYJets10to50_MG"]
TT = ["TTLL_powheg"]
VV = ["WW_pythia", "WZ_pythia", "ZZ_pythia"]
#VV = ["WW_pythia", "ZZ_pythia"]
ST = ["SingleTop_sch_Lep", "SingleTop_tch_top_Incl", "SingleTop_tch_antitop_Incl",
      "SingleTop_tW_top_NoFullyHad", "SingleTop_tW_antitop_NoFullyHad"]
MCList = W + DY + TT + VV + ST

In [12]:
SYSTs = []
if CHANNEL == "MeasFakeEl":
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("ElectronRecoSFUp", "ElectronRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))
if CHANNEL == "MeasFakeMu":
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("MuonRecoSFUp", "MuonRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))

In [13]:
# data
file_path = f"{WORKDIR}/data/MeasFakeRateV2/{ERA}/{CHANNEL}__/DATA/MeasFakeRateV2_{DataStream}.root"
print(file_path)
assert os.path.exists(file_path)
f = ROOT.TFile.Open(file_path)
data = f.Get(f"ZEnriched_{HLTPATH}/{ID}/Central/pair/mass"); data.SetDirectory(0)
f.Close()

/home/choij/workspace/ChargedHiggsAnalysis/data/MeasFakeRateV2/2016preVFP/MeasFakeMu__/DATA/MeasFakeRateV2_DoubleMuon.root


In [14]:
data.Integral()

128916.0

In [15]:
def getSumMC(syst="Central"):
    sum = 0.
    for sample in MCList:
        file_path = f"{WORKDIR}/data/MeasFakeRateV2/{ERA}/{CHANNEL}__RunSyst__/MeasFakeRateV2_{sample}.root"
        assert os.path.exists(file_path)
        f = ROOT.TFile.Open(file_path)
        try:
            h = f.Get(f"ZEnriched_{HLTPATH}/{ID}/{syst}/pair/mass");   h.SetDirectory(0)
            print(sample, h.Integral())
            sum += h.Integral()
        except:
            print(sample)
            continue
    f.Close()
    print(sum)
    return sum

In [16]:
### Get central scale
sumMC = {}
sumMC["Central"] = getSumMC()
for systset in SYSTs:
    if len(systset) == 2:
        systUp, systDown = systset
        sumMC[systUp] = getSumMC(systUp)
        sumMC[systDown] = getSumMC(systDown)
    else:
        syst = systset
        sumMC[syst] = getSumMC(syst)

WJets_MG 869.5875139787346


AssertionError: 

In [61]:
# get scale values
scaleMC = {}
for syst, sum in sumMC.items():
    scaleMC[syst] = data.Integral() / sum
    
diffMC = {}
for syst, sum in scaleMC.items():
    diffMC[syst] = (scaleMC[syst] - scaleMC["Central"]) / scaleMC["Central"]


In [62]:
from pprint import pprint
pprint(diffMC)

{'Central': 0.0,
 'ElectronEnDown': -2.7941238625517497e-06,
 'ElectronEnUp': 1.3132469169650061e-05,
 'ElectronResDown': 0.0,
 'ElectronResUp': 0.0,
 'HeavyTagDownUnCorr': 0.0015102749607894237,
 'HeavyTagUpUnCorr': -0.001497234904265211,
 'JetEnDown': 4.7259781151163166e-05,
 'JetEnUp': -3.063882122540463e-05,
 'JetResDown': 2.159293819600332e-06,
 'JetResUp': 1.141191115781099e-05,
 'L1PrefireDown': -0.007520174564165288,
 'L1PrefireUp': 0.00775920620685325,
 'LightTagDownUnCorr': -0.00040876775394527696,
 'LightTagUpUnCorr': 0.0004086061936796051,
 'MuonEnDown': -7.295800535211774e-05,
 'MuonEnUp': 8.872217937818777e-05,
 'MuonRecoSFDown': 0.0028552176123275127,
 'MuonRecoSFUp': -0.0028430416399660044,
 'PileupReweight': 0.011557422533506511}


In [63]:
print(scaleMC["Central"])

0.5086982702325775
